In [1]:
from torch.utils.data import Dataset, DataLoader
import csv
import json

class Mydataset(Dataset):
    def __init__(self, path):
        x = []
        y = []
        with open(path, newline='') as myFile:  
            reader = csv.reader(myFile)
            rows = list(reader)
        self.x = ["\n".join(json.loads(row[0])[:MAX_LINES]) for row in rows]
        self.y = [row[1] for row in rows]
        self.len = len(self.y)
        self.labels = list(sorted(set(self.y)))
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.len
        
    def get_labels(self):
        return self.labels
    
    def get_label(self, id):
        return self.labels[id]
    
    def get_label_id(self, label):
        return self.labels.index(label)

In [2]:
import string

ALL_LETTERS = string.ascii_letters + string.punctuation + " " + "\n"
N_CHARS = len(ALL_LETTERS)

test_dataset = Mydataset('test_poetry.csv')
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=True)


train_dataset = Mydataset('train_poetry.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

N_LABELS = len(train_dataset.get_labels())

In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

def create_variable(tensor):
    return Variable(tensor)

def make_variables(inputs, labels):
    sequence_and_length = [str2ascii_arr(input) for input in inputs]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths, labels)

def str2ascii_arr(msg):
    arr = [ALL_LETTERS.index(c) for c in msg]
    return arr, len(arr)

def pad_sequences(vectorized_seqs, seq_lengths, labels):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]
    target = labels2tensor(labels)
    if len(labels):
        target = target[perm_idx]
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)
        
def labels2tensor(labels):
    label_ids = [train_dataset.get_label_id(
        label) for label in labels]
    return torch.LongTensor(label_ids)

In [4]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional) + 1
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, input, seq_lengths):
        input = input.t()
        batch_size = input.size(1)
        hidden = self._init_hidden(batch_size)
        embedded = self.embedding(input)
        gru_input = pack_padded_sequence(
            embedded, seq_lengths.data.cpu().numpy())
        self.gru.flatten_parameters()
        output, hidden = self.gru(gru_input, hidden)
        fc_output = self.fc(hidden[-1])
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                             batch_size, self.hidden_size)
        return create_variable(hidden)

In [5]:
def train(i, inputs, labels):
    input, seq_lengths, target = make_variables(inputs, labels)
    output = classifier(input, seq_lengths)
    loss = criterion(output, target)
    classifier.zero_grad()
    loss.backward()
    optimizer.step()
    print('Train Epoch: {}, Batch {} \tLoss: {:.2f}'.format(
        epoch,  i, loss.data[0]))


def evaluate():
    print("Evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)
    for inputs, labels in test_loader:
        input, seq_lengths, target = make_variables(inputs, labels)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))

def test(x):
    input, seq_lengths, target = make_variables([x], [])
    output = classifier(input, seq_lengths)
    pred = output.data.max(1, keepdim=True)[1]
    label_id = pred.cpu().numpy()[0][0]
    print(name, "is", train_dataset.get_label(label_id))

In [ ]:
HIDDEN_SIZE = 100
MAX_LINES = 5
N_LAYERS = 2
BATCH_SIZE = 32
N_EPOCHS = 20

classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_LABELS, N_LAYERS)
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, N_EPOCHS + 1):
    for i, (x, y) in enumerate(train_loader, 1):
        train(i, x, y)
    evaluate()

F:\settings\anaconda\lib\site-packages\ipykernel\__main__.py:9: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1, Batch 1 	Loss: 0.67
Train Epoch: 1, Batch 2 	Loss: 0.73
Train Epoch: 1, Batch 3 	Loss: 0.70
Train Epoch: 1, Batch 4 	Loss: 0.71
Train Epoch: 1, Batch 5 	Loss: 0.71
Train Epoch: 1, Batch 6 	Loss: 0.70
Train Epoch: 1, Batch 7 	Loss: 0.71
Evaluating trained model ...

Test set: Accuracy: 926/1872 (49%)

Train Epoch: 2, Batch 1 	Loss: 0.70
Train Epoch: 2, Batch 2 	Loss: 0.66
Train Epoch: 2, Batch 3 	Loss: 0.69
Train Epoch: 2, Batch 4 	Loss: 0.66
Train Epoch: 2, Batch 5 	Loss: 0.62
Train Epoch: 2, Batch 6 	Loss: 0.63
Train Epoch: 2, Batch 7 	Loss: 0.72
Evaluating trained model ...

Test set: Accuracy: 938/1872 (50%)

Train Epoch: 3, Batch 1 	Loss: 0.63
Train Epoch: 3, Batch 2 	Loss: 0.66
Train Epoch: 3, Batch 3 	Loss: 0.64
Train Epoch: 3, Batch 4 	Loss: 0.64
Train Epoch: 3, Batch 5 	Loss: 0.63
Train Epoch: 3, Batch 6 	Loss: 0.58
Train Epoch: 3, Batch 7 	Loss: 0.66
Evaluating trained model ...

Test set: Accuracy: 993/1872 (53%)

Train Epoch: 4, Batch 1 	Loss: 0.71
Train Epoc